# Telegram - Create Crypto sentiment bot

## Input

### Variables

[Create a Telegram bot and get your API KEY](https://core.telegram.org/bots#6-botfather)

In [1]:
TELEGRAM_API_KEY = "**************"

[Get Newsapi API KEY](https://newsapi.org/)

In [ ]:
NEWSAPI_API_KEY = "****************"

In [ ]:
!pip install python-telegram-bot

In [3]:
import logging
from telegram.ext import *
import numpy as np
import naas_drivers
from datetime import datetime, timedelta

In [4]:
# Set up the logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logging.info('Starting Bot...')

2021-07-23 15:26:33,839 - root - INFO - Starting Bot...


In [6]:
#parameters for newsapi
FROM_DATE = (datetime.now()-timedelta(days=1)).replace(microsecond=0).isoformat()
TO_DATE = datetime.now().replace(microsecond=0).isoformat()

#top marketcap coins
coins= ["bitcoin","etherium", "ripple","dogecoin","cardano", 
        "polygon","binance","polkadot","uniswap","litecoin",
        "chainlink","solana","stellar"]

#connect to Newsapi Naas drivers
naas_drivers.newsapi.connect(NEWSAPI_API_KEY)
naas_drivers.newsapi.connect()

## Model

In [12]:
#user_response
def get_response(user_message):
    
    usr_msg_text = user_message.lower()
    
    if usr_msg_text in coins: #check if returned message has the top coins 
        data = naas_drivers.newsapi.get(q=usr_msg_text,
                                       from_param=FROM_DATE,
                                       to = TO_DATE,
                                       language ="en")
        if len(data)==0:#extra check to returned data
            sentiment = "no news for this ticker in last 24 hours, try another"
            return sentiment
        else:
            sentiment_df =  naas_drivers.sentiment.get(data, column_name="title")#sentiment calculation
            sentiment = sentiment_df["Score"].mean()
    else:
        sentiment = 'There is no data for this ticker\n use \help command'
    
    return sentiment
        

In [16]:
#commands
def start_command(update, context):
    update.message.reply_text('''Hello there! I\'m a crypto sentiment bot.
    Send me the coin name, I\'ll send you sentiment!''')


def help_command(update, context):
    update.message.reply_text('''check the coin names from the list below!
    bitcoin\n etherium\n ripple\n dogecoin\n cardano\n polygon
    binance\n polkadot\n uniswap\n litecoin\n chainlink\n solana\n stellar''')

#message handler
def handle_message(update, context):
    text = str(update.message.text).lower()
    logging.info(f'User ({update.message.chat.id}) says: {text}')

    # Bot response
    sentiment_naas = get_response(text)
    update.message.reply_text(sentiment_naas)

#error logs
def error(update, context):
    # Logs errors
    logging.error(f'Update {update} caused error {context.error}')




In [ ]:
# Run the programme

# create an object "bot"
updater = Updater(TELEGRAM_API_KEY, use_context=True)
dp = updater.dispatcher

#  bot's command handlers
dp.add_handler(CommandHandler('start', start_command))
dp.add_handler(CommandHandler('help', help_command))

# bot's text handlers 
dp.add_handler(MessageHandler(Filters.text, handle_message))

# bot's error handler
dp.add_error_handler(error)

    
# Run the bot
updater.start_polling(1.0)
updater.idle()

2021-07-23 15:34:27,418 - apscheduler.scheduler - INFO - Scheduler started
2021-07-23 15:36:20,023 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-07-23 15:36:20,024 - apscheduler.scheduler - INFO - Scheduler has been shut down
2021-07-23 15:36:21,843 - telegram.ext.updater - WARNING - Exiting immediately!


## Output